This notebook was used to test whether Tensorflow was utilizing our GPU or not.

In [3]:
import tensorflow as tf
import numpy as np
import time

In [7]:
# Set up the dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(10000, 28, 28, 1).astype('float32') / 255
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Define a simple CNN model
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Function to train and time the model
def train_model(device_name):
    with tf.device(device_name):
        model = create_model()
        start_time = time.time()
        model.fit(x_train, y_train, epochs=5, batch_size=64, verbose=0)
        end_time = time.time()
    return end_time - start_time

# Train on CPU
#cpu_time = train_model('/CPU:0')
#print(f'Time taken to train on CPU: {cpu_time:.2f} seconds')

# Train on GPU (if available)
# Train on GPU (if available)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        details = tf.config.experimental.get_device_details(gpu)
        print(f"GPU found: {details['device_name']}")
    gpu_time = train_model('/GPU:0')
    print(f'Time taken to train on GPU: {gpu_time:.2f} seconds')
else:
    print("No GPU found.")


GPU found: NVIDIA GeForce RTX 3050 Ti Laptop GPU


I0000 00:00:1727276885.124999    1004 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


Time taken to train on GPU: 16.72 seconds
